# Problem Statement

In the financial markets, traders have always said, Buy low and Sell High (Motto of the Century)

However, the problem that everyone faces in the investing/trading world is, at any given moment, we are all questioning, is this really the lowest price now and we should buy the stock now / is this really the highest price now and we should sell the stock to take profit or short the stock?

In this project, I am going to solve this problem and identify the reversal points (buy/sell points) for a few selected stocks.

# Scraping Yahoo Financial

In [1]:
import requests
import pandas as pd
import numpy as np
import time
import random

import ta

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
sns.set_style('whitegrid')

Epoch Time 4 years = 126230400

# Initial Data Scraping 

In [3]:
# Get current timestamp (Epoch) Function
def get_current_time():
    current_time = round(time.time(),-2) # current epoch time round of to nearest 100
    four_years_ago = current_time - 126230400 # 59 days ago epoch time
    return current_time, four_years_ago

In [4]:
# Scrape Data from Yahoo
def scrape_yahoo(ticker, current_time, four_years_ago):
    # Create Empty DataFrame
    main_df = pd.DataFrame()
    
    for co in ticker: # Iterate through each ticker
        url = 'https://query1.finance.yahoo.com/v8/finance/chart/{0}?symbol={0}&period1={1}&period2={2}&interval=1d&includePrePost=true&events=div|split|earn&lang=en-SG&region=SG&crumb=JPWLPNPkK0e&corsDomain=sg.finance.yahoo.com'.format(co, int(four_years_ago), int(current_time))
        res=requests.get(url)
        print(url)
        print('-'*40)
        
        # Check Status Code
        if res.status_code != 200:
            print("Error",res.status_code)
            break
        
        # Extract Financial Data
        finance_data = res.json() #Save JSON into dict
        ts = finance_data['chart']['result'][0]['timestamp'] # TimeStamp Data
        closed = finance_data['chart']['result'][0]['indicators']['quote'][0]['close'] # Close Price Data
        opened = finance_data['chart']['result'][0]['indicators']['quote'][0]['open'] # Open Price Data
        low = finance_data['chart']['result'][0]['indicators']['quote'][0]['low'] # Low Price Data
        high = finance_data['chart']['result'][0]['indicators']['quote'][0]['high'] # High Price Data
        volume = finance_data['chart']['result'][0]['indicators']['quote'][0]['volume'] # Volume Data
        localtime = '' # set localtime as null
        
        # Create DataFrame from data
        df = pd.DataFrame({
            'stock': co,
            'time': ts,
            'close_price': closed,
            'open_price': opened,
            'lowest_price': low,
            'highest_price': high,
            'volume': volume,
            'local_time': localtime
            })
        main_df = main_df.append(df).reset_index(drop=True)
        
    return main_df

In [5]:
# Function to create column
def set_time(df, col = 'time'):
    x = time.strftime('%m/%d/%Y %H:%M:%S', time.localtime(df[col]))
    return x

# Cleaning Data

In [6]:
# Clean up DataFrame
def clean_data(data):
    # Drop all rows with volume == 0. We do not need premarket/postmarket data since we cant trade during these hours
    data = data[data['volume']!=0]
    
    # Drop Duplicates
    data = data.drop_duplicates(subset=['stock', 'time'], keep = 'last')
    
    # Drop Null Values
    data = data.dropna()
    
    # Sort by ticker symbol and time
    data = data.sort_values(by = ['stock','time']).reset_index(drop=True)
    
    # Create New Column for localtime
    for i, epoch in enumerate(data['time']):
        if data['local_time'][i] == '':
            data['local_time'][i] = time.strftime('%m/%d/%Y %H:%M:%S', time.localtime(epoch))
            
    # Change local_time datatype from object to date time
    data['local_time'] = pd.to_datetime(data['local_time'])
    
    return data

In [7]:
# Combine Scraped Stocks with current dataset
def scraper(ticker):
    
    # CHECK IF THERE IS ALREADY A CSV FILE
    try:
        all_df = pd.read_csv('../datasets/technical_data.csv')
    except:
        # Create empty dataframe if there is no csv file
        all_df = pd.DataFrame()
        print("No Dataset yet")
    
    # Get Current time and last day of scraping
    current_time, four_years_ago = get_current_time()
    
    # Get DataFrame of Scraped Stocks
    scraped_df = scrape_yahoo(ticker, current_time, four_years_ago)
    
    # Combine current data with scraped data
    all_df = all_df.append(scraped_df).reset_index(drop=True)
    
    # Clean DataFrame
    cleaned_df = clean_data(all_df)
    
    # Save DataFrame to CSV
    cleaned_df.to_csv('../datasets/technical_data.csv', index = False)
    
    return cleaned_df    

In [8]:
# Scrape the Website
ticks = ['AAPL', 'FB', "V", 'INTC', 'DIS', 'CRM', 'NKE', 'GM', 'TWTR', 'TRIP']
df = scraper(ticks)

No Dataset yet
https://query1.finance.yahoo.com/v8/finance/chart/AAPL?symbol=AAPL&period1=1449209000&period2=1575439400&interval=1d&includePrePost=true&events=div|split|earn&lang=en-SG&region=SG&crumb=JPWLPNPkK0e&corsDomain=sg.finance.yahoo.com
----------------------------------------
https://query1.finance.yahoo.com/v8/finance/chart/FB?symbol=FB&period1=1449209000&period2=1575439400&interval=1d&includePrePost=true&events=div|split|earn&lang=en-SG&region=SG&crumb=JPWLPNPkK0e&corsDomain=sg.finance.yahoo.com
----------------------------------------
https://query1.finance.yahoo.com/v8/finance/chart/V?symbol=V&period1=1449209000&period2=1575439400&interval=1d&includePrePost=true&events=div|split|earn&lang=en-SG&region=SG&crumb=JPWLPNPkK0e&corsDomain=sg.finance.yahoo.com
----------------------------------------
https://query1.finance.yahoo.com/v8/finance/chart/INTC?symbol=INTC&period1=1449209000&period2=1575439400&interval=1d&includePrePost=true&events=div|split|earn&lang=en-SG&region=SG&cr

/Users/edwinotw/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,stock,time,close_price,open_price,lowest_price,highest_price,volume,local_time
10055,V,1574778600,182.550003,181.460007,181.210007,182.919998,7079000,2019-11-26 22:30:00
10056,V,1574865000,184.369995,183.000000,182.259995,184.539993,11452700,2019-11-27 22:30:00
10057,V,1575037800,184.509995,183.929993,183.360001,184.850006,3139100,2019-11-29 22:30:00
10058,V,1575297000,181.789993,184.240005,179.869995,184.369995,6883200,2019-12-02 22:30:00
10059,V,1575383400,181.899994,179.899994,179.660004,182.139999,4543300,2019-12-03 22:30:00


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10060 entries, 0 to 10059
Data columns (total 8 columns):
stock            10060 non-null object
time             10060 non-null int64
close_price      10060 non-null float64
open_price       10060 non-null float64
lowest_price     10060 non-null float64
highest_price    10060 non-null float64
volume           10060 non-null int64
local_time       10060 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(2), object(1)
memory usage: 628.8+ KB


# Pre-Processing

## Label Classificaton

### Label Whether the Observation went up or down in Closing Price

In [10]:
# Function to Label whether each observation is a gain/lose/neutral
def label_gain(data, closed, opened):
    df['gain'] = ''
    df.loc[(closed > opened), 'gain'] = 1
    df.loc[(closed < opened), 'gain'] = -1
    df.loc[(closed == opened), 'gain'] = 0
    
    return df

In [11]:
# Creating Column gain
df = label_gain(df, df['close_price'], df['open_price'])

### Label the Target Variable

#### Creating Extra Features to Assist in Labeling the Target Variable

In [12]:
# Create EMA(3) Columns for labeling
df['ema_3'] = ta.trend.ema_indicator(close = df['close_price'], n=3, fillna=True)
df.head()

,stock,time,close_price,open_price,lowest_price,highest_price,volume,local_time,gain,ema_3
0,AAPL,1449239400,119.029999,115.290001,115.110001,119.250000,57777000,2015-12-04 22:30:00,1,119.029999
1,AAPL,1449498600,118.279999,118.980003,117.809998,119.860001,32084200,2015-12-07 22:30:00,-1,118.654999
2,AAPL,1449585000,118.230003,117.519997,116.860001,118.599998,34309500,2015-12-08 22:30:00,1,118.442501
3,AAPL,1449671400,115.620003,117.639999,115.080002,117.690002,46361400,2015-12-09 22:30:00,-1,117.031252
4,AAPL,1449757800,116.169998,116.040001,115.510002,116.940002,29212700,2015-12-10 22:30:00,1,116.600625


In [13]:
# Create Columns for pct_change up to 10 lags
for i in range(1,11):
    df['pct_change_'+str(i)] = df['close_price'].pct_change(-i)*100
df.head()

,stock,time,close_price,open_price,lowest_price,highest_price,volume,local_time,gain,ema_3,pct_change_1,pct_change_2,pct_change_3,pct_change_4,pct_change_5,pct_change_6,pct_change_7,pct_change_8,pct_change_9,pct_change_10
0,AAPL,1449239400,119.029999,115.290001,115.110001,119.250000,57777000,2015-12-04 22:30:00,1,119.029999,0.634089,0.676643,2.949313,2.461910,5.168756,5.823253,7.729207,6.906774,9.221871,12.260681
1,AAPL,1449498600,118.279999,118.980003,117.809998,119.860001,32084200,2015-12-07 22:30:00,-1,118.654999,0.042287,2.300637,1.816304,4.506095,5.156468,7.050413,6.233162,8.533671,11.553334,10.202177
2,AAPL,1449585000,118.230003,117.519997,116.860001,118.599998,34309500,2015-12-08 22:30:00,1,118.442501,2.257395,1.773268,4.461922,5.112020,7.005164,6.188259,8.487796,11.506182,10.155596,10.258323
3,AAPL,1449671400,115.620003,117.639999,115.080002,117.690002,46361400,2015-12-09 22:30:00,-1,117.031252,-0.473440,2.155860,2.791607,4.642959,3.844087,6.092860,9.044614,7.723843,7.824302,6.454288
4,AAPL,1449757800,116.169998,116.040001,115.510002,116.940002,29212700,2015-12-10 22:30:00,1,116.600625,2.641808,3.280579,5.140737,4.338065,6.597536,9.563331,8.236277,8.337214,6.960683,7.534944


I am going to use pct_change, gain, close_price and ema indicator to help label my target variable.

#### Setting Conditions to Label Target Variable

In [14]:
# Conditions to Label whether the point is a postive reversal(1) or not(0)
# Conditions to Label whether the point is a negative reversal(2) or not(0)

signal = '' # Create Empty Signal for buy and sell toggling
df['reversal'] = 0

for index in range(len(df)):
    try:
        if df['stock'][index] == df['stock'][index+2]:
            if signal == "buy": # If toggle is at buy, no more buy signals until first sell signal (2)
                if (df['gain'][index] == -1) & (df['gain'][index-1] == 1):
                    if (df['pct_change_10'][index-1] >= 3.75) or (df['pct_change_8'][index-1] >= 3.75) or (df['pct_change_1'][index-1] >= 3.75):#or (df['pct_change_6'][index-1] >= 0.363.75) or (df['pct_change_3.75'][index-1] >= 0.39) or (df['pct_change_2'][index-1] >= 0.3.751) or (df['pct_change_1'][index-1] >= 0.3.753.75):
                        if df['ema_3'][index+2] > df['close_price'][index+2]:
                            df['reversal'][index-1] = 2
                            signal = 'sell' # Change signal to Sell

            elif signal == 'sell': # If toggle is at sell, no more sell signals until first buy signal(1)
                if (df['gain'][index] == 1) & (df['gain'][index-1] == -1):
                    if (df['pct_change_10'][index-1] <= -3.75) or (df['pct_change_8'][index-1] <= -3.75) or (df['pct_change_1'][index-1] <= -3.75): #or (df['pct_change_6'][index-1] <= -0.363.75) or (df['pct_change_3.75'][index-1] <= -0.39) or (df['pct_change_2'][index-1] <= -0.3.751) or (df['pct_change_1'][index-1] <= -0.3.753.75):
                        if (df['pct_change_10'][index] >= -100) or (df['pct_change_9'][index] >= -100) or (df['pct_change_8'][index] >= -100) or (df['pct_change_7'][index] >= -100) or (df['pct_change_6'][index] >= -100):
                            if df['ema_3'][index+2] < df['close_price'][index+2]:
                                df['reversal'][index-1] = 1
                                signal = 'buy' # Change signal to Buy
            
            else: # At the start where there is no signals yet
                if (df['gain'][index] == -1) & (df['gain'][index-1] == 1):
                    if (df['pct_change_10'][index-1] >= 3.75) or (df['pct_change_8'][index-1] >= 3.75) or (df['pct_change_1'][index-1] >= 3.75):#or (df['pct_change_6'][index-1] >= 0.363.75) or (df['pct_change_3.75'][index-1] >= 0.39) or (df['pct_change_2'][index-1] >= 0.3.751) or (df['pct_change_1'][index-1] >= 0.3.753.75):
                        if df['ema_3'][index+2] > df['close_price'][index+2]:
                            df['reversal'][index-1] = 2
                            signal = 'sell' # Change signal to Sell
                elif (df['gain'][index] == 1) & (df['gain'][index-1] == -1):
                    if (df['pct_change_10'][index-1] <= -3.75) or (df['pct_change_8'][index-1] <= -3.75) or (df['pct_change_1'][index-1] <= -3.75): #or (df['pct_change_6'][index-1] <= -0.363.75) or (df['pct_change_3.75'][index-1] <= -0.39) or (df['pct_change_2'][index-1] <= -0.3.751) or (df['pct_change_1'][index-1] <= -0.3.753.75):
                        if (df['pct_change_10'][index] >= -100) or (df['pct_change_9'][index] >= -100) or (df['pct_change_8'][index] >= -100) or (df['pct_change_7'][index] >= -100) or (df['pct_change_6'][index] >= -100):
                            if df['ema_3'][index+2] < df['close_price'][index+2]:
                                df['reversal'][index-1] = 1
                                signal = 'buy' # Change signal to Buy
                else:
                    df['reversal'][index-1] = 0
            
        else:
            df['reversal'][index-1] = 0
    except:
        print('test')

test


/Users/edwinotw/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/edwinotw/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/edwinotw/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/edwinotw/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is

test
test


In [15]:
# Look at the Baseline Accuracy
df['reversal'].value_counts(normalize = True)

0    0.959245
2    0.020378
1    0.020378
Name: reversal, dtype: float64

### Drop Columns that were Created for Labeling Reversal

For Ease of understanding in the Exploratory Data Analysis portion later, I shall keep the reversal values at [0,1,2]. It will be converted to just [0,1] in the future.

In [16]:
# Drop Columns ['ema_3','pct_change']
df_labeled = df[['stock','time','local_time','open_price','close_price','highest_price','lowest_price','volume','gain','reversal']].copy()
df_labeled.head()

,stock,time,local_time,open_price,close_price,highest_price,lowest_price,volume,gain,reversal
0,AAPL,1449239400,2015-12-04 22:30:00,115.290001,119.029999,119.250000,115.110001,57777000,1,2
1,AAPL,1449498600,2015-12-07 22:30:00,118.980003,118.279999,119.860001,117.809998,32084200,-1,0
2,AAPL,1449585000,2015-12-08 22:30:00,117.519997,118.230003,118.599998,116.860001,34309500,1,0
3,AAPL,1449671400,2015-12-09 22:30:00,117.639999,115.620003,117.690002,115.080002,46361400,-1,0
4,AAPL,1449757800,2015-12-10 22:30:00,116.040001,116.169998,116.940002,115.510002,29212700,1,0


## Save Labeled DataSet to CSV

In [17]:
df_labeled.to_csv('../datasets/labeled_data.csv', index = False)